In [51]:
# ! pip install -q tensorflow_federated

In [52]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/prospice/prospire_FL_shared/MasterFL.py
/kaggle/input/prospice/prospire_FL_shared/train_y_images_rti.npy
/kaggle/input/prospice/prospire_FL_shared/train_x_images_rti.npy
/kaggle/input/prospice/prospire_FL_shared/test_x_images_rti.npy
/kaggle/input/prospice/prospire_FL_shared/readme.txt
/kaggle/input/prospice/prospire_FL_shared/test_y_images_rti.npy
/kaggle/input/prospice/prospire_FL_shared/Parameters.py
/kaggle/input/prospice/prospire_FL_shared/UnetClass.py
/kaggle/input/prospice/prospire_FL_shared/unet_history.json
/kaggle/input/prospice/prospire_FL_shared/fedMasterFLv0.1.ipynb
/kaggle/input/prospice/prospire_FL_shared/.idea/prospire_secon_version_shared.iml
/kaggle/input/prospice/prospire_FL_shared/.idea/.gitignore
/kaggle/input/prospice/prospire_FL_shared/.idea/modules.xml
/kaggle/input/prospice/prospire_FL_shared/.idea/workspace.xml
/kaggle/input/prospice/prospire_FL_shared/.idea/misc.xml
/kaggle/input/prospice/prospire_FL_shared/.idea/vcs.xml
/kaggle/input/prospice/p

In [53]:
import math


class Parameters:

    def __init__(self, dataset):
        self.dataset = dataset

        if dataset == 'wi_outdoor_15_buildings_8_reflections':
            self.power_flies_dir = 'WIOutdoor15Buildings8ReflectionsRxPowerFiles/'
            self.tx_locs_dir = 'WIOutdoor15Buildings8ReflectionsTxRxLocations/txset.txrx'
            self.rx_locs_dir = 'WIOutdoor15Buildings8ReflectionsTxRxLocations/rxset.txrx'
            self.buildings_file = 'WIOutdoor15Buildings8ReflectionsBuildingVertices/ConcreteBuildings.object'
            self.foliage_file = None
            self.xy_grid_rx = True
            self.x_min, self.y_min, self.x_max, self.y_max = 0.0, 0.0, 500.0, 500.0
            self.cell_width = 10.0
            # Total number of TXs is 152 and RXs is 2340
            self.num_sensors_list = range(10, 315, 50)
            # self.num_train_tx_pos_list = range(10, 95, 20)
            self.num_train_tx_pos_list = range(70, 75, 20)
            self.num_train_tx_pos = 90
            self.num_sensors = 200
            self.num_exp_test_tx = 30  # Number of experimental TX positions to test
            self.num_target_locs_for_radio_map = 200
            self.rss_min = -109.0  # Minimum value of acceptable rss
            self.rss_max = -2.0  # Maximum value of acceptable rss
            self.valid_rss = self.rss_min
            self.clip_low_rss = True


In [60]:
import timeit

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout, Flatten, Conv1D, GlobalAveragePooling1D, MaxPooling1D, \
    GlobalMaxPooling1D, LeakyReLU, BatchNormalization, Conv2D, MaxPool2D, Conv2DTranspose, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import load_model
from tensorflow.keras import optimizers
from tensorflow.python.platform import gfile
# import tensorflow.contrib.tensorrt as trt
# from tensorflow.python.framework.graph_util import convert_variables_to_constants
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
from tensorflow.python.platform import gfile
from tensorflow.keras import backend as K
# import KerasModelToFrozenGraph as freeze
import numpy as np
import json, itertools, random
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
import tensorflow as tf
import math, datetime


class UnetClass:

    def __init__(self, params, img_height, img_width, img_depth, var_loss=False, ):
        self.img_height, self.img_width, self.img_depth = img_height, img_width, img_depth
        self.para = params
        self.min_rss = params.rss_min; self.max_rss = params.rss_max
        self.image_preprocessing = None     # 'normalize' will cause problem with custom loss
        if self.image_preprocessing == 'normalize':
            self.datagen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)
        self.normalize_rss = True   # If not using this need to check that predicted values is within range
        self.rescale = 1.0 / (self.max_rss - self.min_rss)
        self.var_loss = var_loss

        self.use_custom_loss = True
        self.model = self.unet_architecture()
        self.history_file = "unet_history.json"; self.checkpoint_model = 'unet_best_model.h5'
        self.model_file = 'unet_model.h5'
        self.best_batch_size, self.best_epochs = 32, 1000    # model parameters for the nn
        self.worker_epochs = 1000, 10
        # self.best_batch_size, self.best_epochs = 32, 1000    # model parameters for the nn
        self.es = EarlyStopping(monitor='val_loss', mode='min', patience=100, verbose=1)    # simple early stopping
        self.mc = ModelCheckpoint(self.checkpoint_model, monitor='val_loss', mode='min', verbose=0,
                                      save_best_only=True) # saves the best model
        self.log_dir = "logs/" + "fit/" + 'Unet' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        self.tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=self.log_dir, histogram_freq=1)

    def unet_architecture(self):
        alpha = 0.0  # for leaky relu

        def conv_block(input, num_filters, kernel_size):
            x = Conv2D(num_filters, kernel_size, padding="same", kernel_initializer='he_normal',
                       kernel_regularizer='l2')(input)
            x = BatchNormalization()(x)
            x = LeakyReLU(alpha=alpha)(x)

            x = Conv2D(num_filters, kernel_size, padding="same", kernel_initializer='he_normal',
                       kernel_regularizer='l2')(x)
            x = BatchNormalization()(x)
            x = LeakyReLU(alpha=alpha)(x)
            return x

        def encoder_block(input, num_filters, kernel_size):
            # kernel_size = (3, 3)
            x = conv_block(input, num_filters, kernel_size)
            p = MaxPool2D((2, 2))(x)
            # x = BatchNormalization()(x)
            # p = Dropout(rate=0.25)(p)
            return x, p

        def decoder_block(input, skip_features, num_filters, kernel_size):
            x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same", kernel_initializer='he_normal',
                                kernel_regularizer='l2')(input)
            # x = BatchNormalization()(x)
            # x = LeakyReLU(alpha=alpha)(x)
            x = Concatenate()([skip_features, x])
            # kernel_size = (3, 3)
            x = conv_block(x, num_filters, kernel_size)
            return x

        inputs = Input(shape=(self.img_height - 2, self.img_width - 2, self.img_depth)) # for adjustment from 50 to 48

        # Encoder
        s1, p1 = encoder_block(inputs, 32, (3, 3))
        s2, p2 = encoder_block(p1, 64, (3, 3))
        s3, p3 = encoder_block(p2, 128, (3, 3))
        s4, p4 = encoder_block(p3, 256, (3, 3))

        b1 = conv_block(p4, 256, (3, 3))

        d1 = decoder_block(b1, s4, 256, (3, 3))
        d2 = decoder_block(d1, s3, 128, (3, 3))
        d3 = decoder_block(d2, s2, 64, (3, 3))
        d4 = decoder_block(d3, s1, 32, (3, 3))

        outputs = Conv2D(1, (1, 1), padding="same", activation="relu")(d4)
        model = Model(inputs, outputs, name="U-Net")
        adam = optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999)
        if self.use_custom_loss:
            model.compile(loss=self.custom_loss, optimizer='adam')
        else:
            model.compile(optimizer='adam', loss="mean_absolute_error")
            # model.compile(optimizer='adam', loss="mean_squared_error")
        # model.summary()
        return model

    def custom_loss(self, y_true, y_pred):
        img_height = y_true.shape[1]; img_width = y_true.shape[2]

        if self.var_loss:
            lambda_overestimate = 1.0
            lambda_underestimate = 10.0
        else:
            lambda_overestimate = 1.0
            lambda_underestimate = 1.0

        y_true_mask = tf.where(y_true < 0.0, 0.0, 1.0)
        y_true_mask_flattened = K.reshape(y_true_mask, (-1, img_height * img_width))
        y_true_count_nonzero = K.sum(y_true_mask_flattened, axis=-1)

        diff_loss_masked = (y_true - y_pred) * y_true_mask

        underestimate_loss_mask = tf.where(diff_loss_masked > 0.0, 1.0 * lambda_underestimate, 0.0)
        underestimate_loss_mask_flattened = K.reshape(underestimate_loss_mask, (-1, img_height * img_width))
        underestimate_count_nonzero = K.sum(underestimate_loss_mask_flattened, axis=-1)

        overestimate_loss_mask = tf.where(diff_loss_masked < 0.0, 1.0 * lambda_overestimate, 0.0)
        overestimate_loss_mask_flattened = K.reshape(overestimate_loss_mask, (-1, img_height * img_width))
        overestimate_count_nonzero = K.sum(overestimate_loss_mask_flattened, axis=-1)

        diff_loss_masked = tf.where(diff_loss_masked > 0.0, diff_loss_masked * lambda_underestimate, diff_loss_masked)
        diff_loss_masked = tf.where(diff_loss_masked < 0.0, diff_loss_masked * lambda_overestimate, diff_loss_masked)
        abs_loss_masked = tf.abs(diff_loss_masked)
        loss_masked = abs_loss_masked

        loss_masked = K.reshape(loss_masked, (-1, img_height * img_width))
        sum_loss_masked = K.sum(loss_masked, axis=-1)
        mean_loss_masked = sum_loss_masked / (underestimate_count_nonzero + overestimate_count_nonzero)
        # mean_loss_masked = sum_loss_masked / y_true_count_nonzero

        return mean_loss_masked

    def training(self, train_x, train_y, start_training, worker_epochs = -1):
        if not start_training:
            self.model = load_model(self.model_file, custom_objects={'custom_loss': self.custom_loss})
            return

        if self.normalize_rss:
            if self.use_custom_loss:
                train_y = np.where(train_y == 0.0, self.min_rss - 10.0, train_y)
            else:
                train_y = np.where(train_y == 0.0, self.min_rss, train_y)
            train_y = train_y - self.min_rss
            train_y = train_y * self.rescale
        # print(train_x.shape, train_y.shape)
        train_x = np.transpose(train_x, [0, 2, 3, 1])   # for channel last orientation
        train_y = train_y.reshape(tuple(list(train_y.shape) + [1]))
        # print(train_x.shape, train_y.shape)
        # remove the outermost rows and columns for easier integration with UNET structure
        train_x = train_x[:, 1:-1, 1:-1, :]; train_y = train_y[:, 1:-1, 1:-1, :]
        print('Train and test shape for UNET', train_x.shape, train_y.shape)
        print('Train and test set data size', train_x.nbytes, train_y.nbytes)

        if worker_epochs != -1:
            self.best_epochs = worker_epochs

        if self.image_preprocessing == 'normalize':
            train_idxs = np.random.choice(np.arange(train_x.shape[0]), size=int(train_x.shape[0] * 0.8), replace=False)
            val_idxs = list(set(np.arange(train_x.shape[0])) - set(train_idxs))

            x_val, y_val = train_x[val_idxs, ...], train_y[val_idxs, ...]
            train_x, train_y = train_x[train_idxs, ...], train_y[train_idxs, ...]
            print('Train_x and train_y shape for UNET', train_x.shape, train_y.shape)
            print('val_x and val_y shape for UNET', x_val.shape, y_val.shape)

            self.datagen.fit(train_x)

            history = self.model.fit(self.datagen.flow(train_x, train_y, batch_size=self.best_batch_size),
                                     steps_per_epoch=math.ceil(len(train_x) / float(self.best_batch_size)),
                                     epochs=self.best_epochs,
                                     callbacks=[self.es, self.mc],
                                     validation_data=self.datagen.flow(x_val, y_val, batch_size=self.best_batch_size),
                                     validation_steps=math.ceil(len(x_val) / float(self.best_batch_size)),
                                     verbose=0)

        else:
            history = self.model.fit(train_x, train_y,
                                     validation_split=0.2,
                                     batch_size=self.best_batch_size,
                                     epochs=self.best_epochs,
                                     callbacks=[self.es, self.mc, self.tensorboard_callback],
                                     verbose=1,
                                     shuffle=True)

        if self.use_custom_loss:
            self.model = load_model(self.checkpoint_model, custom_objects={'custom_loss': self.custom_loss})
        else:
            self.model.load_weights(self.checkpoint_model)  # load the saved best model
        self.model.save(self.model_file)        # Save the model for future use
        with open(self.history_file, "w") as json_file:     # save the history in a file
            json.dump(history.history, json_file)
#         print('best_val_loss, best_train_loss', self.get_train_stats())

    # Returns the validation loss and train loss for the chosen model
    def get_train_stats(self):
        with open(self.history_file, "r") as json_file:
            history = json.load(json_file)
        best_val_loss = min(history['val_loss'])
        i, = np.where(np.array(history['val_loss']) == best_val_loss)
        best_idx = i[0]
        best_train_loss = history['loss'][best_idx]
        return best_val_loss, best_train_loss

    def predict_rss(self, test_x, batch=False):
        if len(test_x.shape) == 3:
            test_x = test_x.reshape(tuple([1] + list(test_x.shape)))
        test_x = np.transpose(test_x, [0, 2, 3, 1])     # for making channels last
        test_x = test_x[:, 1:-1, 1:-1, :]       # for making images 48x48

        if self.image_preprocessing == 'normalize':
            test_iterator = self.datagen.flow(test_x, None, batch_size=1)
            test_x = test_iterator.next()

        tx = timeit.default_timer()
        if batch:
            predicted_values = self.model.predict(test_x, batch_size=1024, verbose=0)
        else:
            # predicted_values = self.model.predict(test_x, verbose=0)
            predicted_values = self.model(test_x, training=False)
        # print(timeit.default_timer() - tx)

        if self.normalize_rss:
            rss_values = predicted_values / self.rescale
            rss_values = rss_values + self.min_rss
        else:
            rss_values = predicted_values
        rss_values = np.where(test_x[..., 1] == 0.0, 0.0, rss_values[..., 0])  # use predictions only for active RX locs
        # make the images back to 50x50
        temp_res = np.zeros((rss_values.shape[0], self.img_height, self.img_width))
        temp_res[:, 1:-1, 1:-1] = rss_values
        rss_values = temp_res
        if not batch:
            return rss_values[0, ...]
        else:
            return rss_values

In [61]:
import numpy as np
import timeit, time
# from UnetClass import UnetClass
# from Parameters import Parameters
from matplotlib import pyplot as plt

dataset = 'wi_outdoor_15_buildings_8_reflections'  # dataset to use. For other available datasets, see Parameters.py
para = Parameters(dataset)
num_pixels_x = int((para.x_max - para.x_min) / para.cell_width)  # image width
num_pixels_y = int((para.y_max - para.y_min) / para.cell_width)  # image height
# for sampling RXs and creating images
num_samples = 40
num_images = 200

gen_plots = False
use_aug = False

start_training = True
n_workers = 4 # number of workers to use for federated learning :: 1 means traditional learning without FL

def augmentation(train_x_images, train_y_images):
    train_x_images_aug = []; train_y_images_aug = []
    for i in range(train_x_images.shape[0]):
        tx_img = train_x_images[i, 0, ...]
        rx_img = train_x_images[i, 1, ...]
        map_img = train_x_images[i, 2, ...]
        rti_img = train_x_images[i, 3, ...]
        rss_img = train_y_images[i]

        non_zero_pixels = np.nonzero(rx_img)
        non_zero_pixels = np.array(list(zip(non_zero_pixels[0], non_zero_pixels[1])))

        for j in range(num_images):
            selected_pixels = np.random.choice(non_zero_pixels.shape[0], size=num_samples, replace=False)
            selected_pixels = non_zero_pixels[selected_pixels, :]
            selected_pixels = tuple(list(zip(*selected_pixels)))

            rx_img_new = np.zeros_like(rx_img)
            rx_img_new[selected_pixels] = rx_img[selected_pixels]
            rti_img_new = np.zeros_like(rti_img)
            rti_img_new[selected_pixels] = rti_img[selected_pixels]
            new_data = np.stack([tx_img, rx_img_new, map_img, rti_img_new], axis=0)
            train_x_images_aug.append(new_data)

            rss_img_new = np.zeros_like(rss_img)
            rss_img_new[selected_pixels] = rss_img[selected_pixels]
            train_y_images_aug.append(rss_img_new)

            # plotting some sample images
            if gen_plots:
                fig, axs = plt.subplots(nrows=2, ncols=5, constrained_layout=True)
                plot_data = [tx_img, rx_img, map_img, rti_img, rss_img, new_data[0, ...], new_data[1, ...],
                             new_data[2, ...], new_data[3, ...], rss_img_new]
                titles = ['TX', 'RX', 'Map', 'RTI', 'RSS', 'TX', 'RX_samp', 'Map', 'RTI_samp', 'RSS_samp']
                for idx, ax in enumerate(axs.flat):
                    ax.grid(True)
                    vec = plot_data[idx]; title = titles[idx]

                    im = ax.imshow(vec, aspect='auto', cmap='viridis', interpolation="nearest")
                    cbar = fig.colorbar(im, ax=ax, orientation="vertical", shrink=0.99, aspect=40, pad=0.01)
                    cbar.ax.tick_params()

                    ax.set_xticks(np.arange(0, 50, 5) - 0.5); ax.set_xticklabels(10 * np.arange(0, 50, 5), rotation=45)
                    ax.set_yticks(np.arange(0, 50, 5) + 0.5); ax.set_yticklabels(10 * np.flip(np.arange(0, 50, 5)))
                    # ax.tick_params(labelsize=labelsize)
                    ax.set_title(title)
                plt.show()
    return [np.array(train_x_images_aug), np.array(train_y_images_aug)]

def federated_averaging(global_model, worker_models, weights):
    global_weights = global_model.get_weights()
    num_workers = len(worker_models)
    
    # Initialize averaged weights
    new_weights = [np.zeros_like(w) for w in global_weights]
    
    # Weighted average of local models' weights
    for i, worker_model in enumerate(worker_models):
        local_weights = worker_model.get_weights()
        for j in range(len(new_weights)):
            new_weights[j] += local_weights[j] * weights[i]
    
    # Set the averaged weights to the global model
    global_model.set_weights(new_weights)
    # return global_model

class FederatedWorker:
    def __init__(self, worker_id, nunet_obj, train_x_images, train_y_images, params, worker_epochs=100, total_workers=4):
        self.worker_id = worker_id
        self.nunet_obj = nunet_obj
        self.model = nunet_obj.model
        self.train_x_images = train_x_images * gen_mask(train_x_images, n=total_workers)
        self.train_y_images = train_y_images
        self.worker_epochs = worker_epochs
        self.params = params
    
    def gen_mask(tensor, prob=None, n=None, dtype=torch.float32):
        """
        batched mask 
        is all one for idx 0,3 as per 0 is TX and 3 is location map
        and for idx 1 and 2 is 1 for sampled points with probability of 1/n or prob as as prompted
        """
        if (prob is not None) and (n is not None):
            return -1
        if n is not None:
            prob = 1 / n
        mask = np.ones(tensor.shape)
        if len(tensor.shape) == 4:
            for i in range(tensor.shape[0]):
                mask[i, 1] = np.random.choice([0, 1], size=tensor.shape[2:], p=[1-prob, prob])
                mask[i, 3] = np.random.choice([0, 1], size=tensor.shape[2:], p=[1-prob, prob])
        else:
            raise ValueError("Tensor must be a 4D batch of 3D tensors.")
        return torch.tensor(mask, dtype=dtype)

    def train_local_model(self):
        # Train local model and return training time, val_loss, and train_loss
        start_time = time.time()
        self.nunet_obj.training(self.train_x_images, self.train_y_images, start_training=True, worker_epochs=self.worker_epochs)
        training_time = time.time() - start_time
#         val_loss, train_loss = self.model.get_train_stats()
#         return val_loss, train_loss, training_time
    

def main():
    train_x_images, train_y_images = np.load('/kaggle/input/prospice/prospire_FL_shared/train_x_images_rti.npy', 'r'), np.load('/kaggle/input/prospice/prospire_FL_shared/train_y_images_rti.npy', 'r')
    test_x_images, test_y_images = np.load('/kaggle/input/prospice/prospire_FL_shared/test_x_images_rti.npy', 'r'), np.load('/kaggle/input/prospice/prospire_FL_shared/test_y_images_rti.npy', 'r')
    
    train_x_images_tensor = torch.tensor(train_x_images, dtype=torch.float32)

#     train_x_images, train_y_images = np.array_split(r_train_x_images, n_workers), np.array_split(r_train_y_images, n_workers)
    
    print(f'number of sub-data : {len(train_y_images)}; ', 
          'train_x_images[0].shape : ', train_x_images[0].shape, '; train_y_images[0].shape : ', train_y_images[0].shape, 
          '; \ntest_x_images.shape : ', test_x_images.shape, '; test_y_images.shape : ', test_y_images.shape)    

    if use_aug:
        train_x_images, train_y_images = augmentation(train_x_images, train_y_images)
        test_x_images, test_y_images = augmentation(test_x_images, test_y_images)
        print('Post augmentation:')
        print('train_x_images.shape, train_y_images.shape, test_x_images.shape, test_y_images.shape',
              train_x_images.shape, train_y_images.shape, test_x_images.shape, test_y_images.shape)

    global_nunet_obj_rti = UnetClass(para, num_pixels_x, num_pixels_y, train_x_images[-1].shape[1], False)
    workers_nunet_obj_rti = [UnetClass(para, num_pixels_x, num_pixels_y, train_x_images[-1].shape[1], False) for i in range(n_workers)]

    # intialize workers
    w_epcochs = 1
    workers = [FederatedWorker(i, workers_nunet_obj_rti[i], train_x_images_tensor, train_y_images, para, worker_epochs = w_epcochs, total_workers = n_workers) for i in range(n_workers)]
    epochs = 1 #int(1000//100) # UnetClass.best_epochs
    for epoch in range(epochs):
        print(f'Epoch {epoch+1}')
        
        # Train local models
        for worker in workers:
            worker.train_local_model()
        
        # Federated averaging
        federated_averaging(global_nunet_obj_rti.model, [worker_obj.model for worker_obj in workers],  [1/n_workers]*n_workers)

    # compute train error
    temp_pred = global_nunet_obj_rti.predict_rss(train_x_images, True)
    temp_err = np.where(train_y_images == 0.0, train_y_images, np.abs(temp_pred - train_y_images))
    train_error = (np.sum(temp_err)) / np.count_nonzero(temp_err)
    print('train_error', train_error)
    # prediction
    nunet_rti_prediction = []
    for item, vals in zip(test_x_images, test_y_images):
        temp_unet = global_nunet_obj_rti.predict_rss(item)
        nunet_rti_prediction.append(temp_unet)
    nunet_rti_prediction = np.reshape(np.array(nunet_rti_prediction), test_y_images.shape)
    temp_err = np.where(test_y_images == 0.0, test_y_images, np.abs(nunet_rti_prediction - test_y_images))
    print('test error', np.sum(temp_err) / np.count_nonzero(temp_err))


if __name__ == "__main__":
    main()

## main code base ends here

number of sub-data : 70;  train_x_images[0].shape :  (4, 50, 50) ; train_y_images[0].shape :  (50, 50) ; 
test_x_images.shape :  (30, 4, 50, 50) ; test_y_images.shape :  (30, 50, 50)


ValueError: The filepath provided must end in `.keras` (Keras model format). Received: filepath=unet_best_model.h5

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

train_x_images, train_y_images = np.load('/kaggle/input/prospice/prospire_FL_shared/train_x_images_rti.npy', 'r'), np.load('/kaggle/input/prospice/prospire_FL_shared/train_y_images_rti.npy', 'r')
test_x_images, test_y_images = np.load('/kaggle/input/prospice/prospire_FL_shared/test_x_images_rti.npy', 'r'), np.load('/kaggle/input/prospice/prospire_FL_shared/test_y_images_rti.npy', 'r')
# train_x_images, train_y_images = np.array_split(r_train_x_images, n_workers), np.array_split(r_train_y_images, n_workers)

# print(f'number of sub-data : {len(train_y_images)}; ', 
#       'train_x_images[0].shape : ', train_x_images.shape, '; train_y_images[0].shape : ', train_y_images.shape, 
#       '; \ntest_x_images.shape : ', test_x_images.shape, '; test_y_images.shape : ', test_y_images.shape)    

# print((train_x_images[0][0]))
# print((train_x_images[0][1]))
# print((train_x_images[0][2]))
# print((train_x_images[0][3]))

# for tensor in train_x_images[0]:
#     plt.imshow(tensor, cmap='viridis')
#     plt.colorbar()
#     plt.title('Tensor Image')
#     plt.show()


def gen_mask(tensor, prob=None, n=None, dtype=torch.float32):
    if (prob is not None) and (n is not None):
        return -1
    if n is not None:
        prob = 1 / n
        
    mask = np.ones(tensor.shape)
    
    # Generate a mask for the 2nd and 4th elements along the second dimension for each tensor in the batch
    if len(tensor.shape) == 4:
        for i in range(tensor.shape[0]):
            mask[i, 1] = np.random.choice([0, 1], size=tensor.shape[2:], p=[1-prob, prob])
            mask[i, 3] = np.random.choice([0, 1], size=tensor.shape[2:], p=[1-prob, prob])
    else:
        raise ValueError("Tensor must be a 4D batch of 3D tensors.")
    
    return torch.tensor(mask, dtype=dtype)

# Step 1: Create a batch of 3D tensors
batch_size = 100
p = torch.tensor(np.random.rand(batch_size, 4, 50, 50), dtype=torch.float32)

for tenso in tensor[0][1:2]:
    plt.imshow(p[0][2], cmap='viridis')
    plt.colorbar()
    plt.title('Tensor Image')
    plt.show()
    break
n = 2
mask_tensor = gen_mask(tensor, n=n)
sampled_tensor = tensor * mask_tensor

# print("Original Tensor:")
# print(tensor)
# print("\nMask:")
# print(mask_tensor)
# print("\nSampled Tensor:")
# print(sampled_tensor)
for pa in tensor[0][1:2]:
    plt.imshow(sampled_tensor[0][1], cmap='viridis')
    plt.colorbar()
    plt.title('Tensor Image')
    plt.show()
    break